# dropdown menu Michaelis-Menten inhibitor troubleshooting
__objective__: develop an interactive visualization for looking at Michaelis-Menten inhibition (i.e. varying [I] and Ki), but make it so users can select which inhibition type they want to visualize from a dropdown menu

__approach__:
1. study a dropdown menu example from the Boekh gallery
* get a deeper understanding of the callback syntax with dropdown menus
* re-organize inhibitor visualization code to allow for drop down menu interface

---
## gallery example of a dropdown menu
__python server example__<br>
tried the following example: https://github.com/bokeh/bokeh/tree/master/examples/app/crossfilter
* however, when you edit this to allow for display inline, it does not work
* this is because this is meant to be run on a python server - it uses python to control the interactive visualization on the plot, so it must be run within a Bokeh Server (no javascript directly involved)
* we do not have the option to run a bokeh server (easily) - so it's best to see if I can do these sorts of visualizations using javascript alone

    import pandas as pd

    from bokeh.layouts import column, row
    from bokeh.models import Select
    from bokeh.palettes import Spectral5
    from bokeh.plotting import curdoc, figure
    from bokeh.sampledata.autompg import autompg_clean as df

    df = df.copy()

    SIZES = list(range(6, 22, 3))
    COLORS = Spectral5
    N_SIZES = len(SIZES)
    N_COLORS = len(COLORS)

    # data cleanup
    df.cyl = df.cyl.astype(str)
    df.yr = df.yr.astype(str)
    del df['name']

    columns = sorted(df.columns)
    discrete = [x for x in columns if df[x].dtype == object]
    continuous = [x for x in columns if x not in discrete]

    def create_figure():
        xs = df[x.value].values
        ys = df[y.value].values
        x_title = x.value.title()
        y_title = y.value.title()

        kw = dict()
        if x.value in discrete:
            kw['x_range'] = sorted(set(xs))
        if y.value in discrete:
            kw['y_range'] = sorted(set(ys))
        kw['title'] = "%s vs %s" % (x_title, y_title)

        p = figure(plot_height=600, plot_width=800, tools='pan,box_zoom,hover,reset', **kw)
        p.xaxis.axis_label = x_title
        p.yaxis.axis_label = y_title

        if x.value in discrete:
            p.xaxis.major_label_orientation = pd.np.pi / 4

        sz = 9
        if size.value != 'None':
            if len(set(df[size.value])) > N_SIZES:
                groups = pd.qcut(df[size.value].values, N_SIZES, duplicates='drop')
            else:
                groups = pd.Categorical(df[size.value])
            sz = [SIZES[xx] for xx in groups.codes]

        c = "#31AADE"
        if color.value != 'None':
            if len(set(df[color.value])) > N_COLORS:
                groups = pd.qcut(df[color.value].values, N_COLORS, duplicates='drop')
            else:
                groups = pd.Categorical(df[color.value])
            c = [COLORS[xx] for xx in groups.codes]

        p.circle(x=xs, y=ys, color=c, size=sz, line_color="white", alpha=0.6, hover_color='white', hover_alpha=0.5)

        return p


    def update(attr, old, new):
        layout.children[1] = create_figure()


    x = Select(title='X-Axis', value='mpg', options=columns)
    x.on_change('value', update)

    y = Select(title='Y-Axis', value='hp', options=columns)
    y.on_change('value', update)

    size = Select(title='Size', value='None', options=['None'] + continuous)
    size.on_change('value', update)

    color = Select(title='Color', value='None', options=['None'] + continuous)
    color.on_change('value', update)

    controls = column(x, y, color, size, width=200)
    layout = row(controls, create_figure())

    curdoc().add_root(layout)
    curdoc().title = "Crossfilter"

it looks like the figure above works by creating a function (<code>create_fiture</code>) which generates the whole plot - for the dropdown menus, the above function is called, via the <code>update</code> function, to re-create the whole figure to change the X and Y axis values

... this would be very hard to do via JS

__pure js example__<br>
I also found this example, using pure JS, which makes 2x widgets communicate with one another:

In [ ]:
from bokeh.io import output_notebook
output_notebook()

In [ ]:
from bokeh.io import show
from bokeh.layouts import column, row
from bokeh.models import RadioButtonGroup, CustomJS, MultiSelect

opts = {
    0: ["foo", "bar"],
    1: ["baz", "quux"],
}

bg = RadioButtonGroup(labels=["Stuff", "Things"], active=0)

ms = MultiSelect(options=opts[0])

bg.js_on_change('active', CustomJS(args=dict(ms=ms), code="""
    const opts = %s
    ms.options = opts[cb_obj.active]
""" % opts))

show(row(ms, bg))

I think I may be able to do the example I'm thinking of it I could write functions in Javascript, and change which function is called ... to do that, I'm goning to learn a little bit of JavaScript

---
## javascript notes
https://www.w3schools.com/js/

### JS HOME
* JavaScript is 1 of 3x core components in a web developer's toolkit:
    - HTML: holds content in web pages
    - CSS: formats how webpages are laid out
    - JS: allows for interaction with webpages (i.e. programs their behavior)
    
### JS Introduction
* JS can be embeded pretty much wherever in an HTML document (or be housed separately) and can exert effects on pretty much any element in the document
* JS can change the links behind pages, displayed content, which components are shown, etc.
* JS can interact with CSS in addition to HTML
* double and single quotes can be used in JS

### JS Statements
* __statement__: an instruction that a computer interprets and executes
> var x; //Statement 1<br>
x = 5; //Statement 2

* a "program" is a list of statements
* in JS, statements are separated by <code>;</code>, in contrast to Python, which uses new lines, however, semicolons are not necessarily required
* multiple statements on 1x line are allowed:
> a = 5; b = 6; c = 7;
* whitespace in JS is ignored - adding whitespace to make the document more readibly is A-OK! e.g. these two statements are equal:
>var person = "hedge";<br>
var person ="hedge";
* whitespace around operators is encouraged
* linebreaks are likewise used to make code more readible; if a line is too long, add between operators]
> document.getElementById("demo").innerHTML =<br>
"Hello Dolly!";
* __code blocks__ are segments of code that are meant to be ran at the same time
* curly brackets { } are used to designate code blocks - e.g. in function calls
        function myFunction() {
            document.getElementById("demo1").innerHTML = "Hello Dolly!";
            document.getElementById("demo2").innerHTML = "How are You?";
* __JavaScript Keywords__ are special words that are indicate the action that JS should perform - these are reserved within the namespace, and therefore cannot be used as variable names, etc.
> e.g. <code>function</code> denotes a function definition, <code>if ... else</code> marks blocks of code that have flow control, <code>var</code> declares a variable

### JS Snytax
* There are 2x value types in JS: literals and variables
* __literals__ - are fixed values that directly represent the data, e.g. string and numbers
* string literals are written between double or single quotes
> "this is ok"<br>
'this is equally ok'
* numbers are written as integers or with decimals
> 10<br>
> 10.124
* __variables__ - are not fixed, but assigned to certain fixed value types; variables are names that can be used to referenced the underlying data associated with the variable
* the <code>var</code> keyword is used to declare variables, then <code>=</code> is used to assign a declared variable to a value - note how multiple variables can be defined on a single line
> var x,y;<br>
x = 6;<br>
y = 5;
* __operators__ are objects take a defined input, perform a certain task, and return an output in a similar way to functions, except operators differ in terms of syntax (e.g. in arithmetic operators, like +, take the inputs on either side of the operator, instead of a list of inputs being passed to a function)
* JS operators (+ - * / =) work just like python
> var x;<br>
x = 5;<br>
(x + 6) / 11;
* an __expression__ is a combined set of inputs/operators/functions that combine into a single output
> (x * 10) - 5
* an __evaluation__ is the process of interpreting an expression down to its output
* __comments__ are lines of code that aren't executed by the computer, but are included by programmers to help explain their code
* comments are included after <code>//</code> until the end of the line, or between <code>/*</code> and <code>*/</code> across mutliple lines 
> var x = 5; // this text is not executed<br>
// var y = 6; this whole line isn't executed
* __identifiers__ are names of different objects that can be defined in code, e.g. variables, functions, etc.
* identifiers must start with either a letter, underscore, or dollar sign; subsequent letters can be letters, numbers, an underscore, or a dollar sign
> var x, hello_there, myVar // these are all acceptable identifiers<br>
> 1x // this is not
* JS programmers like to use use lower camel case for variable names, however
>lowerCamelCase<br>
anotherLowerCamelCase

### JS Variables
* as discussed above, the <code>var</code> keyword is used to declare variables, and the <code>=</code> operator is used to assign variables to values
* variables can be declared and assigned across different lines
> var x;<br>
x = 6;
* variables can be declared and assigned in the same line
> var x = 6;
* multiple variables can be declared/assigned on the same line - note the usage of commas
> var x = 6, y = 10, z = "volvo";
* or across multiple lines:
> var x = 6, <br>y = 10,<br>z = "volvo";
* variables can be defined without an assigned value, so that users can assign them later
* strings can be concatenated
> "John" + " " + "Doe"

### JS Operators
see here for a table of JS operators:<br>
https://www.w3schools.com/js/js_operators.asp
* these include arithmetic operators, along with different kinds of assignment operators (e.g. +=)
* also these include __comparison operators__ (operators used to evaluate Boolean equations) like == and !=
* logical operators __&&__ (AND), __||__(OR), etc.
* operators also include different symbols to denote membership, function calls, etc.
> <code>.</code> denotes membership person.name <br>
<code>[]</code> also denotes membership person.[name]<br>
<code>()</code> denotes a function call myFunc(5)
* see the bottom of the following page for a list of operators used across JS
https://www.w3schools.com/js/js_arithmetic.asp

### JS Data Types
* datatypes are important because they allow functions to perform computations on data, determine how to combine inputs, and check if inputs make sense at all
* there are main data types in JS are: strings, numbers, objects, Booleans, Arrays, and undefined
* __strings__ are text data between double or single quotes
* __numbers__ are numeric data that is of integer or float quantities
* __objects__ are like dictionaries in python - associated key value pairs and are written in between curly brackets
>  var person = {firstName:"John", lastName:"Doe", age:50, eyeColor:"blue"};
* __boleans__ dictate Boolean values used with conditional operators and are either <code>true</code> or <code>false</code> - note case
* __Arrays__ are ordered sequences of data, they are dictated by square brackets; these are 0-based, just like python
> var car = ["BMW", "volvo", "saab"]
* data types are __dynamic__ in JS, i.e. the data type of a variable can change over the course of code execution
> var x; //x is undefined<br>
x = 5; //x is numeric<br>
x = 'hello' //x is a string

### JS Functions
* functions are blocks of code that are executed at the same time
* JS function syntax:
        function myFunction(p1, p2) {
           return p1 * p2;   // The function returns the product of p1 and p2
           }
* functions are defined by the __function__ keyword
* the function keyword is followed by __parentheses__ which contain the __parameters__ of the function separated by commas
* parameters exist in the function definition, arguments are the actual values passed to the function
* then, the body of the function is denoted by __curly brackets__ 
* the __return__ keyword denotes the __return statement__, which dictates what values the function gives as its output
* functions are invocked by passing arguments to them in between parenthases
        myFunction(2, 5)
* __variable scope__ determines which variables are recognized throughout the code; within functions, variables are __local__ to the function, i.e. they cannot be accessed from outside the function, just like python    

### JS Conditions
* __conditional statements__ allow for certain blocks of code to execute dependent on the outcome of a Boolean operation (i.e. a certain condition in the code)
* conditional statement operators:
> if<br> else if<br> else<br> switch
* conditional statement syntax in JS
        if (condition1) {
          //  block of code to be executed if condition1 is true
        } else if (condition2) {
          //  block of code to be executed if the condition1 is false and condition2 is true
        } else {
          //  block of code to be executed if the condition1 is false and condition2 is false
        }

idea for simple function in javascript to contrl plotting:
    # mock function, covert to JS, that'll allow you to plot certain types
    button_val = 0
    x = [1,2,3]
    y = [2,3,4]

    def inhib1(x):
        pass

    def inhib2(x):
        pass

    if button_val == 0:
        y = inhib1(x)

    elif button_val == 1:
        y = inhib2(x)

    # change the outside variable
 
---
## simple javascript test run

In [ ]:
# import libraries
import os
import sys
import numpy as np

from bokeh.layouts import row, column
from bokeh.models import CustomJS, Slider, Label, Span
from bokeh.plotting import figure, output_file, show, ColumnDataSource

root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)

from pharmaplot import mm

In [ ]:
from bokeh.io import show
from bokeh.layouts import column
from bokeh.models import RadioButtonGroup, CustomJS, MultiSelect, Select

opts = {
    0: ["foo", "bar"],
    1: ["baz", "quux"],
    2: ["other"]
}

bg = RadioButtonGroup(labels=["Stuff", "Things", "other"], active=0)

ms = MultiSelect(options=opts[0])

bg.js_on_change('active', CustomJS(args=dict(ms=ms), code="""
    const opts = %s
    ms.options = opts[cb_obj.active]
""" % opts))

show(row(ms, bg))

In [ ]:
# define data
x = np.linspace(0, 10, 500)
y = np.sin(x)

source = ColumnDataSource(data=dict(x=x, y=y))

# create data and plot initial line
plot = figure(y_range=(-10, 10), plot_width=400, plot_height=400)

plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

# define sliders
amp_slider = Slider(start=0.1, end=10, value=1, step=.1, title="Amplitude")
freq_slider = Slider(start=0.1, end=10, value=1, step=.1, title="Frequency")
phase_slider = Slider(start=0, end=6.4, value=0, step=.1, title="Phase")
offset_slider = Slider(start=-5, end=5, value=0, step=.1, title="Offset")

# define button
#bg = RadioButtonGroup(labels=["Stuff", "Things"], active=0)
bg = Select(title="Inhibition Type:", value="foo", options=["foo", "bar", "baz"])

# define callback
callback = CustomJS(args=dict(source=source, 
                              amp=amp_slider, 
                              freq=freq_slider, 
                              phase=phase_slider, 
                              offset=offset_slider,
                              bg=bg),
                    code="""
    const data = source.data;
    const A = amp.value;
    const k = freq.value;
    const phi = phase.value;
    const B = offset.value;
    const x = data['x'];
    const y = data['y'];
    const b = bg.value;
    
    function sin(x, k, phi){
        return Math.sin(k*x+phi);
    }
    function cos(x, k, phi){
        return Math.cos(k*x+phi)
    }
    for (var i = 0; i < x.length; i++) {
        if (b == "foo"){
            y[i] = B + A*sin(x[i],k, phi);
        } else if (b == "bar"){
            y[i] = B + A*cos(x[i],k, phi);
        } else if (b == "baz"){
            y[i] = B + A*x[i]*k*phi
        }
        
    }
    source.change.emit();
""")

amp_slider.js_on_change('value', callback)
freq_slider.js_on_change('value', callback)
phase_slider.js_on_change('value', callback)
offset_slider.js_on_change('value', callback)
bg.js_on_change('value', callback)

# layout and visualize
layout = row(
    plot,
    column(amp_slider, freq_slider, phase_slider, offset_slider, bg),
)

show(layout)

holy fuck I got it!

Select option allows you to select a certain value
using the .value attribute, I can pass that value to the custom callback function

I set each of the different types of line as a function

Then use conditional statments to call each function specifically!!!!!!!!!!

---
## generating inhibitor plots

In [ ]:
# generate data for plotting
log_start = -1
log_end = 4

vmax = 100
km = 5
ki = 1
conc_i = 0

x_line = np.logspace(log_start, log_end, num=100)
y_line = mm.mm_competitive(x_line, vmax=vmax, km=km, ki=ki, conc_i=conc_i)

x_points = np.logspace(log_start, log_end, num=20)
y_points = mm.mm_competitive(x_points, vmax=vmax, km=km, ki=ki, conc_i=conc_i)

# set up source data and plot lines that will vary
line_source = ColumnDataSource(data=dict(x=x_line, y=y_line))
point_source = ColumnDataSource(data=dict(x=x_points, y=y_points))

plot = figure(y_range=(-5, 120), x_range=(-5, 100), plot_width=600, plot_height=400, 
              x_axis_label='[S]: substrate concentration (μM)',
              y_axis_label='initial velocity (μM/s)',
              title='Michaelis-Menten Kinetics with Inhibition')

plot.line('x', 'y', source=line_source, line_width=3, line_alpha=0.6, color='black')
plot.circle('x', 'y', source=point_source, size=10, color='black')

# set up static line and annotations
plot.line(x_line, y_line, line_width=5, color='blue', line_alpha=0.3)
plot.circle(x_points, y_points, size=10, color='blue', line_alpha=0.3)

mytext = Label(x=10, y=87, text='[I] = 0 (μM)', 
               text_color="blue", text_alpha=0.5)
plot.add_layout(mytext)

# add axes lines
vline = Span(location=0, dimension='height', line_color='black', line_width=1, line_alpha=0.3)
hline = Span(location=0, dimension='width', line_color='black', line_width=1, line_alpha=0.3)
plot.renderers.extend([vline, hline])

# set up java script callback function and widgets to make plot interactive
ci_slider = Slider(start=0, end=100, value=0, step=1, title="[I] (μM)")
ki_slider = Slider(start=1, end=100, value=50, step=1, title="Ki (μM)")
inhib_select = Select(title="Inhibition Type:", value="competitive", 
                      options=["competitive", "noncompetitive", "uncompetitive"])

callback = CustomJS(args=dict(LineSource=line_source, 
                              PointSource=point_source,
                              ci=ci_slider, 
                              ki=ki_slider,
                              vmax=vmax,
                              km=km,
                              inhibType=inhib_select),
                    code="""
    const LineData = LineSource.data;
    const PointData = PointSource.data;
    const VMAX = vmax;
    const KM = km;
    const CI = ci.value;
    const KI = ki.value;
    const lx = LineData['x'];
    const ly = LineData['y'];
    const px = PointData['x'];
    const py = PointData['y'];
    const it = inhibType.value;
    
    function competitive(x, VMAX, KM, CI, KI){
        return (VMAX*x)/((KM*(1+(CI/KI))+x));
    }
    function noncompetitive(x, VMAX, KM, CI, KI){
        return (VMAX*x)/((KM*(1+(CI/KI)))+(x*(1+(CI/KI))));
    }
    function uncompetitive(x, VMAX, KM, CI, KI){
        return (VMAX*x)/(KM+(x*(1+(CI/KI))));
    }
    for (var i = 0; i < lx.length; i++) {
        if (it == "competitive"){
            ly[i] = competitive(lx[i], VMAX, KM, CI, KI);
            py[i] = competitive(px[i], VMAX, KM, CI, KI);
        } else if (it == "noncompetitive"){
            ly[i] = noncompetitive(lx[i], VMAX, KM, CI, KI);
            py[i] = noncompetitive(px[i], VMAX, KM, CI, KI);
        } else if (it == "uncompetitive"){
            ly[i] = uncompetitive(lx[i], VMAX, KM, CI, KI);
            py[i] = uncompetitive(px[i], VMAX, KM, CI, KI);      
        }
    }

    LineSource.change.emit();
    PointSource.change.emit();
    // (VMAX*lx[i])/((KM*(1+(CI/KI)))+(lx[i]*(1+(CI/KI)))) noncompetitive
    // (VMAX*lx[i])/(KM+(lx[i]*(1+(CI/KI)))) uncompetitive
""")

# add sliders to plot and display
ci_slider.js_on_change('value', callback)
ki_slider.js_on_change('value', callback)
inhib_select.js_on_change('value', callback)

layout = row(
    plot,
    column(ci_slider, ki_slider, inhib_select),
)

#output_file("mm.html", title="mm.py example")
show(layout)

GOT IT!!!!!!!

plan worked - VICTORY!